Task-1 (mPAP)

In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Reproducibility
def set_seed(seed_value=123):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(123)

# Lead paths and label
lead_file_paths = {
    "LEAD_I": "../../Data_processing/data_aspire_PAP_1/LEAD_I.pt",
    "LEAD_II": "../../Data_processing/data_aspire_PAP_1/LEAD_II.pt",
    "LEAD_III": "../../Data_processing/data_aspire_PAP_1/LEAD_III.pt",
    "LEAD_aVR": "../../Data_processing/data_aspire_PAP_1/LEAD_aVR.pt",
    "LEAD_aVL": "../../Data_processing/data_aspire_PAP_1/LEAD_aVL.pt",
    "LEAD_aVF": "../../Data_processing/data_aspire_PAP_1/LEAD_aVF.pt",
    "LEAD_V1": "../../Data_processing/data_aspire_PAP_1/LEAD_V1.pt",
    "LEAD_V2": "../../Data_processing/data_aspire_PAP_1/LEAD_V2.pt",
    "LEAD_V3": "../../Data_processing/data_aspire_PAP_1/LEAD_V3.pt",
    "LEAD_V4": "../../Data_processing/data_aspire_PAP_1/LEAD_V4.pt",
    "LEAD_V5": "../../Data_processing/data_aspire_PAP_1/LEAD_V5.pt",
    "LEAD_V6": "../../Data_processing/data_aspire_PAP_1/LEAD_V6.pt"
}
labels_file_path = "../../Data_processing/data_aspire_PAP_1/labels.pt"

# Load data
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Validate shape
sample_count = len(next(iter(ecg_lead_tensors.values())))
assert len(labels) == sample_count
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count

# Lead configuration
use_6_leads = True
lead_names = (
    ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVL", "LEAD_aVF"]
    if use_6_leads else list(lead_file_paths.keys())
)

# Dataset class
class ECGMultiLeadDatasetWithLabels(Dataset):
    def __init__(self, ecg_leads, labels, lead_names):
        self.ecg_leads = {lead: ecg_leads[lead] for lead in lead_names}
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        lead_data = [self.ecg_leads[lead][idx].unsqueeze(1) for lead in self.ecg_leads]
        stacked = torch.cat(lead_data, dim=1)  # shape: (seq_len, num_leads)
        label = self.labels[idx]
        return stacked, label

dataset = ECGMultiLeadDatasetWithLabels(ecg_lead_tensors, labels, lead_names)


Using device: cuda


In [2]:
import math
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=2048):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)  # even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # odd indices

        pe = pe.unsqueeze(0)  # shape: (1, max_len, d_model)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)
class ECGTransformerClassifier(nn.Module):
    def __init__(self, input_channels, seq_len, d_model=128, nhead=8, num_layers=4, dim_feedforward=512, num_classes=2, dropout=0.1):
        super(ECGTransformerClassifier, self).__init__()
        self.embedding = nn.Linear(input_channels, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, num_classes)
        )

    def forward(self, x):
        # Input shape: (B, seq_len, num_leads)
        x = self.embedding(x)  # (B, seq_len, d_model)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)  # (B, seq_len, d_model)
        x = x.mean(dim=1)  # Global average pooling
        out = self.classifier(x)  # (B, num_classes)
        return out


In [ ]:
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

def evaluate_model(model, loader):
    model.eval()
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)
    return acc, auc, f1, mcc
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\nFOLD {fold + 1}")
    print('-----------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=16, shuffle=False)

    seq_len = dataset[0][0].shape[0]  # shape = (seq_len, num_leads)
    model = ECGTransformerClassifier(
        input_channels=len(lead_names),
        seq_len=seq_len,
        d_model=64,
        nhead=4,
        num_layers=2,
        dim_feedforward=256,
        num_classes=2,
        dropout=0.1
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    train_classifier(model, train_loader, criterion, optimizer, epochs=50)
    metrics = evaluate_model(model, test_loader)
    print(f"Fold {fold+1} Results: Accuracy: {metrics[0]:.4f}, AUC: {metrics[1]:.4f}, F1: {metrics[2]:.4f}, MCC: {metrics[3]:.4f}")
    fold_results.append(metrics)

# Final metrics
accuracies, aucs, f1s, mccs = zip(*fold_results)
print("\nFinal Results:")
print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Mean AUC:      {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"Mean F1:       {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
print(f"Mean MCC:      {np.mean(mccs):.4f} ± {np.std(mccs):.4f}")


Task-2: PAWP

In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Reproducibility
def set_seed(seed_value=123):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(123)

# Define file paths for the 12-lead ECG data and labels
lead_file_paths = {
    "LEAD_I": "../../Data_processing/data_aspire_PAWP_1/LEAD_I.pt",
    "LEAD_II": "../../Data_processing/data_aspire_PAWP_1/LEAD_II.pt",
    "LEAD_III": "../../Data_processing/data_aspire_PAWP_1/LEAD_III.pt",
    "LEAD_aVR": "../../Data_processing/data_aspire_PAWP_1/LEAD_aVR.pt",
    "LEAD_aVL": "../../Data_processing/data_aspire_PAWP_1/LEAD_aVL.pt",
    "LEAD_aVF": "../../Data_processing/data_aspire_PAWP_1/LEAD_aVF.pt",
    "LEAD_V1": "../../Data_processing/data_aspire_PAWP_1/LEAD_V1.pt",
    "LEAD_V2": "../../Data_processing/data_aspire_PAWP_1/LEAD_V2.pt",
    "LEAD_V3": "../../Data_processing/data_aspire_PAWP_1/LEAD_V3.pt",
    "LEAD_V4": "../../Data_processing/data_aspire_PAWP_1/LEAD_V4.pt",
    "LEAD_V5": "../../Data_processing/data_aspire_PAWP_1/LEAD_V5.pt",
    "LEAD_V6": "../../Data_processing/data_aspire_PAWP_1/LEAD_V6.pt"
}
labels_file_path = "../../Data_processing/data_aspire_PAWP_1/labels.pt"

# Load data
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Validate shape
sample_count = len(next(iter(ecg_lead_tensors.values())))
assert len(labels) == sample_count
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count

# Lead configuration
use_6_leads = True
lead_names = (
    ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVL", "LEAD_aVF"]
    if use_6_leads else list(lead_file_paths.keys())
)

# Dataset class
class ECGMultiLeadDatasetWithLabels(Dataset):
    def __init__(self, ecg_leads, labels, lead_names):
        self.ecg_leads = {lead: ecg_leads[lead] for lead in lead_names}
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        lead_data = [self.ecg_leads[lead][idx].unsqueeze(1) for lead in self.ecg_leads]
        stacked = torch.cat(lead_data, dim=1)  # shape: (seq_len, num_leads)
        label = self.labels[idx]
        return stacked, label

dataset = ECGMultiLeadDatasetWithLabels(ecg_lead_tensors, labels, lead_names)


Using device: cuda


In [ ]:
def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

def evaluate_model(model, loader):
    model.eval()
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)
    return acc, auc, f1, mcc
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\nFOLD {fold + 1}")
    print('-----------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=16, shuffle=False)

    seq_len = dataset[0][0].shape[0]  # shape = (seq_len, num_leads)
    model = ECGTransformerClassifier(
        input_channels=len(lead_names),
        seq_len=seq_len,
        d_model=64,
        nhead=4,
        num_layers=2,
        dim_feedforward=256,
        num_classes=2,
        dropout=0.1
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    train_classifier(model, train_loader, criterion, optimizer, epochs=50)
    metrics = evaluate_model(model, test_loader)
    print(f"Fold {fold+1} Results: Accuracy: {metrics[0]:.4f}, AUC: {metrics[1]:.4f}, F1: {metrics[2]:.4f}, MCC: {metrics[3]:.4f}")
    fold_results.append(metrics)

# Final metrics
accuracies, aucs, f1s, mccs = zip(*fold_results)
print("\nFinal Results:")
print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Mean AUC:      {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"Mean F1:       {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
print(f"Mean MCC:      {np.mean(mccs):.4f} ± {np.std(mccs):.4f}")


Task-3

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
# File paths
lead_file_paths = {
    "LEAD_I": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_I.pt",
    "LEAD_II": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_II.pt",
    "LEAD_III": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_III.pt",
    "LEAD_aVR": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVR.pt",
    "LEAD_aVL": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVL.pt",
    "LEAD_aVF": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_aVF.pt",
    "LEAD_V1": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V1.pt",
    "LEAD_V2": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V2.pt",
    "LEAD_V3": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V3.pt",
    "LEAD_V4": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V4.pt",
    "LEAD_V5": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V5.pt",
    "LEAD_V6": "D:/ukbiobank/ECG_PAWP_UKB_Final/LEAD_V6.pt"
}
labels_file_path = "D:/ukbiobank/ECG_PAWP_UKB_Final/labels.pt"

# Load all lead tensors and labels
ecg_lead_tensors = {lead: torch.load(path) for lead, path in lead_file_paths.items()}
labels = torch.load(labels_file_path)

# Ensure all leads have the same number of samples as the labels
sample_count = len(next(iter(ecg_lead_tensors.values())))
print(len(labels))
print(sample_count)
assert len(labels) == sample_count, "Mismatch between number of labels and samples."
for tensor in ecg_lead_tensors.values():
    assert len(tensor) == sample_count, "All leads must have the same number of samples."

# Lead configuration
use_6_leads = True
lead_names = (
    ["LEAD_I", "LEAD_II", "LEAD_III", "LEAD_aVR", "LEAD_aVL", "LEAD_aVF"]
    if use_6_leads else list(lead_file_paths.keys())
)

# Dataset class
class ECGMultiLeadDatasetWithLabels(Dataset):
    def __init__(self, ecg_leads, labels, lead_names):
        self.ecg_leads = {lead: ecg_leads[lead] for lead in lead_names}
        self.labels = labels

    def __len__(self):
        return len(next(iter(self.ecg_leads.values())))

    def __getitem__(self, idx):
        lead_data = [self.ecg_leads[lead][idx].unsqueeze(1) for lead in self.ecg_leads]
        stacked = torch.cat(lead_data, dim=1)  # shape: (seq_len, num_leads)
        label = self.labels[idx]
        return stacked, label

dataset = ECGMultiLeadDatasetWithLabels(ecg_lead_tensors, labels, lead_names)

16416
16416


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, matthews_corrcoef

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


def train_classifier(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * labels.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

def evaluate_model(model, loader):
    model.eval()
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    f1 = f1_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)
    return acc, auc, f1, mcc
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(labels)), labels)):
    print(f"\nFOLD {fold + 1}")
    print('-----------------------------')

    train_subset = Subset(dataset, train_ids)
    test_subset = Subset(dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

    seq_len = dataset[0][0].shape[0]  # shape = (seq_len, num_leads)
    print("training")
    model = ECGTransformerClassifier(
        input_channels=len(lead_names),
        seq_len=seq_len,
        d_model=64,
        nhead=4,
        num_layers=2,
        dim_feedforward=256,
        num_classes=2,
        dropout=0.1
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    train_classifier(model, train_loader, criterion, optimizer, epochs=50)
    metrics = evaluate_model(model, test_loader)
    print(f"Fold {fold+1} Results: Accuracy: {metrics[0]:.4f}, AUC: {metrics[1]:.4f}, F1: {metrics[2]:.4f}, MCC: {metrics[3]:.4f}")
    fold_results.append(metrics)

# Final metrics
accuracies, aucs, f1s, mccs = zip(*fold_results)
print("\nFinal Results:")
print(f"Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Mean AUC:      {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"Mean F1:       {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
print(f"Mean MCC:      {np.mean(mccs):.4f} ± {np.std(mccs):.4f}")
